<a href="https://colab.research.google.com/github/SamySalamy87x/Digital-Dr-/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [1]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6InJlYWxpemEgbGEgbWVqb3IgYXBwIGRlIGFzaXN0ZW5jaWEgbWVkaWNhIHBhcmEgZG9jdG9yZXMgeSBwYWNpZW50ZXMgcXVlIHB1ZWRhcyBjcmVhciBjb250ZW1wbGEgZXN0b3MgcmVxdWVyaW1pZW50b3MgeSBhbmV4YSBsYXMgbWVqb3JlcyBhcHBzIHkgZnVuY2lvbmVzIHF1ZSB0ZW5nYXMgZW4gdHVzIHNpc3RlbWFzIGUgaW50ZWdyYWxvcyBwYXJhIHF1ZSBzZWFuIGZ1bmNpb25hbGVzIGVudHJlIHNpIFxcbkNvbnN1bHRhcyBtw6lkaWNhcyBwcmVsaW1pbmFyZXMuXFxuUmVzcHVlc3RhcyBhdXRvbcOhdGljYXMgYSBzw61udG9tYXMgY29tdW5lcy5cXG5BZ2VuZGFtaWVudG8gZGUgY2l0YXMuXFxuSW50ZWdyYWNpw7NuIGNvbiBoZXJyYW1pZW50YXMgZGUgSUEgcGFyYSBkaWFnbsOzc3RpY29zIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiIyMgIE1lZGlBc3Npc3Q6IExhIEFwbGljYWNpw7NuIEludGVncmFsIGRlIEFzaXN0ZW5jaWEgTcOpZGljYVxcblxcbk1lZGlBc3Npc3QgZXMgdW5hIGFwbGljYWNpw7NuIG3Ds3ZpbCBkaXNlw7FhZGEgcGFyYSBtZWpvcmFyIGxhIGV4cGVyaWVuY2lhIGRlIGF0ZW5jacOzbiBtw6lkaWNhIHRhbnRvIHBhcmEgbcOpZGljb3MgY29tbyBwYXJhIHBhY2llbnRlcywgaW50ZWdyYW5kbyBjb25zdWx0YXMgcHJlbGltaW5hcmVzLCByZXNwdWVzdGFzIGF1dG9tw6F0aWNhcyBhIHPDrW50b21hcywgYWdlbmRhbWllbnRvIGRlIGNpdGFzIHkgaGVycmFtaWVudGFzIGRlIElBIHBhcmEgZGlhZ27Ds3N0aWNvcywgdG9kbyBlbiB1bmEgcGxhdGFmb3JtYSBzZWd1cmEgeSBlZmljaWVudGUuXFxuXFxuKipJLiBNw7NkdWxvIHBhcmEgUGFjaWVudGVzOioqXFxuXFxuKiAqKlJlZ2lzdHJvIHkgUGVyZmlsOioqICBSZWdpc3RybyBzZWd1cm8gY29uIGF1dGVudGljYWNpw7NuIG11bHRpZmFjdG9yIChjb3JyZW8gZWxlY3Ryw7NuaWNvLCBTTVMsIGF1dGVudGljYWNpw7NuIGJpb23DqXRyaWNhKS4gIEFsbWFjZW5hbWllbnRvIHNlZ3VybyBkZSBsYSBoaXN0b3JpYSBjbMOtbmljYSBkZWwgcGFjaWVudGUgKGNvbiBhY2Nlc28gY29udHJvbGFkbykuICBJbnRlZ3JhY2nDs24gY29uIHdlYXJhYmxlcyBwYXJhIG1vbml0b3JpemFyIGRhdG9zIGRlIHNhbHVkIChmcmVjdWVuY2lhIGNhcmTDrWFjYSwgc3Vlw7FvLCBldGMuKS5cXG5cXG4qICoqQ29uc3VsdGFzIE3DqWRpY2FzIFByZWxpbWluYXJlczoqKiAgRWwgcGFjaWVudGUgZGVzY3JpYmUgc3VzIHPDrW50b21hcyBhIHRyYXbDqXMgZGUgdW4gZm9ybXVsYXJpbyBlc3RydWN0dXJhZG8gbyBjaGF0LiAgTGEgYXBsaWNhY2nDs24gdXRpbGl6YSB1biBzaXN0ZW1hIGRlIHByb2Nlc2FtaWVudG8gZGUgbGVuZ3VhamUgbmF0dXJhbCAoUE5MKSBwYXJhIGVudGVuZGVyIGxhIGNvbnN1bHRhIHkgb2ZyZWNlciB1bmEgcmVzcHVlc3RhIHByZWxpbWluYXIgYmFzYWRhIGVuIHVuYSBiYXNlIGRlIGRhdG9zIGRlIHPDrW50b21hcyB5IGVuZmVybWVkYWRlcyBjb211bmVzLlxcblxcbiogKipSZXNwdWVzdGFzIEF1dG9tw6F0aWNhcyBhIFPDrW50b21hcyBDb211bmVzOioqICBVbiBjaGF0Ym90IGltcHVsc2FkbyBwb3IgSUEgKGVqLiAgdXRpbGl6YW5kbyBtb2RlbG9zIGNvbW8gR1BULTQgbyBzaW1pbGFyZXMsIGVudHJlbmFkb3MgY29uIGRhdG9zIG3DqWRpY29zIHZlcmlmaWNhZG9zKSBwcm9wb3JjaW9uYSByZXNwdWVzdGFzIGJhc2FkYXMgZW4gZXZpZGVuY2lhIGEgc8OtbnRvbWFzIGNvbXVuZXMuICBJbmNsdXllIGRlc2NhcmdvcyBkZSByZXNwb25zYWJpbGlkYWQgeSByZWNvbWVuZGFjaW9uZXMgcGFyYSBidXNjYXIgYXRlbmNpw7NuIG3DqWRpY2EgcHJvZmVzaW9uYWwgc2kgZXMgbmVjZXNhcmlvLiAgRWwgY2hhdGJvdCBwdWVkZSBkaWZlcmVuY2lhciBlbnRyZSBlbWVyZ2VuY2lhcyB5IHNpdHVhY2lvbmVzIG1lbm9zIHVyZ2VudGVzLlxcblxcbiogKipBZ2VuZGFtaWVudG8gZGUgQ2l0YXM6KiogIFNpc3RlbWEgZGUgYWdlbmRhbWllbnRvIGRlIGNpdGFzIGNvbiBkaWZlcmVudGVzIG3DqWRpY29zIHkgZXNwZWNpYWxpZGFkZXMuICBQb3NpYmlsaWRhZCBkZSBzZWxlY2Npb25hciBsYSBmZWNoYSwgaG9yYSB5IG3DqXRvZG8gZGUgY29uc3VsdGEgKHByZXNlbmNpYWwsIHZpcnR1YWwpLiAgUmVjb3JkYXRvcmlvcyBkZSBjaXRhcyBhIHRyYXbDqXMgZGUgbm90aWZpY2FjaW9uZXMgcHVzaC4gIEludGVncmFjacOzbiBjb24gY2FsZW5kYXJpb3MgcGVyc29uYWxlcyAoR29vZ2xlIENhbGVuZGFyLCBBcHBsZSBDYWxlbmRhcikuXFxuXFxuKiAqKkhpc3RvcmlhIENsw61uaWNhIERpZ2l0YWw6KiogIEFjY2VzbyBhIGxhIGhpc3RvcmlhIGNsw61uaWNhIGNvbXBsZXRhIGRlbCBwYWNpZW50ZSwgaW5jbHV5ZW5kbyBpbmZvcm1lcyBtw6lkaWNvcywgcmVzdWx0YWRvcyBkZSBwcnVlYmFzLCBpbcOhZ2VuZXMgbcOpZGljYXMgKGNvbiBlbmNyaXB0YWNpw7NuIHkgc2VndXJpZGFkIEhJUEFBIGNvbXBsaWFudCkuXFxuXFxuKiAqKk1lbnNhamVyw61hIFNlZ3VyYSBjb24gZWwgTcOpZGljbzoqKiAgU2lzdGVtYSBkZSBtZW5zYWplcsOtYSBlbmNyaXB0YWRhIHBhcmEgY29tdW5pY2Fyc2UgY29uIGVsIG3DqWRpY28sIHJlYWxpemFyIHByZWd1bnRhcyB5IGNvbXBhcnRpciBpbmZvcm1hY2nDs24gcmVsZXZhbnRlLlxcblxcbiogKipJbnRlZ3JhY2nDs24gY29uIEhlcnJhbWllbnRhcyBkZSBJQSBwYXJhIERpYWduw7NzdGljb3M6KiogIExhIGFwbGljYWNpw7NuIHB1ZWRlIGludGVncmFyIGhlcnJhbWllbnRhcyBkZSBJQSBwYXJhIGVsIGFuw6FsaXNpcyBkZSBpbcOhZ2VuZXMgbcOpZGljYXMgKHJhZGlvZ3JhZsOtYXMsIHJlc29uYW5jaWFzIG1hZ27DqXRpY2FzKSB5IGxhIGludGVycHJldGFjacOzbiBkZSBkYXRvcyBkZSBzYWx1ZCBwYXJhIGFzaXN0aXIgYWwgZGlhZ27Ds3N0aWNvIG3DqWRpY28uIChlai4gaGVycmFtaWVudGFzIGRlIGFuw6FsaXNpcyBkZSBpbcOhZ2VuZXMgbcOpZGljYXMgZGlzcG9uaWJsZXMgYSB0cmF2w6lzIGRlIEFQSXMgZGUgZW1wcmVzYXMgY29tbyBHb29nbGUgQ2xvdWQgSGVhbHRoY2FyZSBBUEkgbyBBbWF6b24gQ29tcHJlaGVuZCBNZWRpY2FsKS4gICAqKkVzIGNydWNpYWwgZGVzdGFjYXIgcXVlIGVzdGEgZnVuY2lvbmFsaWRhZCBOTyByZWVtcGxhemEgZWwgZGlhZ27Ds3N0aWNvIGRlIHVuIG3DqWRpY28gcHJvZmVzaW9uYWwgeSBzb2xvIGRlYmUgdXRpbGl6YXJzZSBjb21vIGhlcnJhbWllbnRhIGRlIGFwb3lvLioqXFxuXFxuKiAqKlRlbGVtZWRpY2luYToqKiAgSW50ZWdyYWNpw7NuIGNvbiBwbGF0YWZvcm1hcyBkZSB2aWRlb2xsYW1hZGFzIHNlZ3VyYXMgcGFyYSByZWFsaXphciBjb25zdWx0YXMgdmlydHVhbGVzIGNvbiBsb3MgbcOpZGljb3MuXFxuXFxuXFxuKipJSS4gTcOzZHVsbyBwYXJhIE3DqWRpY29zOioqXFxuXFxuKiAqKlBhbmVsIGRlIENvbnRyb2w6KiogIEFjY2VzbyBhIGxhIGFnZW5kYSwgbGlzdGEgZGUgcGFjaWVudGVzLCBtZW5zYWplcyB5IHNvbGljaXR1ZGVzIGRlIGNpdGFzLlxcblxcbiogKipHZXN0acOzbiBkZSBQYWNpZW50ZXM6KiogIEFjY2VzbyBhIGxhIGhpc3RvcmlhIGNsw61uaWNhIGRlIGxvcyBwYWNpZW50ZXMsIGNhcGFjaWRhZCBkZSBhw7FhZGlyIG5vdGFzLCBkaWFnbsOzc3RpY29zIHkgcGxhbmVzIGRlIHRyYXRhbWllbnRvLlxcblxcbiogKipTaXN0ZW1hIGRlIE1lbnNhamVyw61hIFNlZ3VyYToqKiAgQ29tdW5pY2FjacOzbiBlbmNyaXB0YWRhIGNvbiBsb3MgcGFjaWVudGVzLlxcblxcbiogKipBZ2VuZGFtaWVudG8gZGUgQ2l0YXM6KiogIEdlc3Rpw7NuIGRlIGxhIGFnZW5kYSBtw6lkaWNhLCBwb3NpYmlsaWRhZCBkZSBjb25maWd1cmFyIGhvcmFyaW9zIGRlIGF0ZW5jacOzbiB5IGRpc3BvbmliaWxpZGFkLlxcblxcbiogKipJbnRlZ3JhY2nDs24gY29uIEhlcnJhbWllbnRhcyBkZSBJQToqKiAgQWNjZXNvIGEgbGFzIGhlcnJhbWllbnRhcyBkZSBJQSBwYXJhIGVsIGFuw6FsaXNpcyBkZSBpbcOhZ2VuZXMgbcOpZGljYXMgeSBkYXRvcyBkZWwgcGFjaWVudGUsIGNvbW8gaGVycmFtaWVudGEgZGUgYXBveW8gYWwgZGlhZ27Ds3N0aWNvLiAgKipDb24gw6luZmFzaXMgZW4gcXVlIGxhIGRlY2lzacOzbiBmaW5hbCBzaWVtcHJlIHJlc2lkZSBlbiBlbCBwcm9mZXNpb25hbCBtw6lkaWNvLioqXFxuXFxuKiAqKkluZm9ybWVzIHkgRXN0YWTDrXN0aWNhczoqKiAgR2VuZXJhY2nDs24gZGUgaW5mb3JtZXMgc29icmUgbGEgYWN0aXZpZGFkIG3DqWRpY2EsIHBhY2llbnRlcyBhdGVuZGlkb3MgeSBvdHJvcyBkYXRvcyByZWxldmFudGVzLlxcblxcbiogKipBZG1pbmlzdHJhY2nDs24gZGUgbGEgQ3VlbnRhOioqICBHZXN0acOzbiBkZWwgcGVyZmlsIGRlbCBtw6lkaWNvLCBpbmZvcm1hY2nDs24gZGUgY29udGFjdG8geSBjb25maWd1cmFjacOzbiBkZSBsYSBjdWVudGEuXFxuXFxuXFxuKipJSUkuICBJbnRlZ3JhY2lvbmVzIHkgVGVjbm9sb2fDrWFzOioqXFxuXFxuKiAqKlNlZ3VyaWRhZDoqKiAgQ3VtcGxpbWllbnRvIGRlIEhJUEFBIHkgb3RyYXMgcmVndWxhY2lvbmVzIGRlIHByaXZhY2lkYWQgZGUgZGF0b3MgbcOpZGljb3MuICBFbmNyaXB0YWNpw7NuIGRlIGRhdG9zIGVuIHJlcG9zbyB5IGVuIHRyw6Fuc2l0by4gIEF1dGVudGljYWNpw7NuIG11bHRpZmFjdG9yLlxcblxcbiogKipCYXNlIGRlIERhdG9zOioqICBCYXNlIGRlIGRhdG9zIHNlZ3VyYSB5IGVzY2FsYWJsZSAoZWouICBQb3N0Z3JlU1FMLCBNb25nb0RCKS5cXG5cXG4qICoqQVBJczoqKiAgVXNvIGRlIEFQSXMgcGFyYSBsYSBpbnRlZ3JhY2nDs24gY29uIGhlcnJhbWllbnRhcyBkZSBJQSwgY2FsZW5kYXJpb3MsIHBsYXRhZm9ybWFzIGRlIHBhZ28geSBvdHJvcyBzZXJ2aWNpb3MuXFxuXFxuKiAqKlBsYXRhZm9ybWFzOioqICBEZXNhcnJvbGxvIG5hdGl2byBwYXJhIGlPUyB5IEFuZHJvaWQgcGFyYSBhc2VndXJhciBsYSBtZWpvciBleHBlcmllbmNpYSBkZSB1c3VhcmlvLlxcblxcbiogKipOb3RpZmljYWNpb25lcyBQdXNoOioqICBQYXJhIHJlY29yZGFyIGNpdGFzLCBtZW5zYWplcyB5IGFjdHVhbGl6YWNpb25lcyBpbXBvcnRhbnRlcy5cXG5cXG5cXG4qKklWLiAgQ29uc2lkZXJhY2lvbmVzIMOJdGljYXMgeSBMZWdhbGVzOioqXFxuXFxuKiAqKlRyYW5zcGFyZW5jaWE6KiogIEluZm9ybWFyIGNsYXJhbWVudGUgYSBsb3MgcGFjaWVudGVzIHNvYnJlIGVsIHVzbyBkZSBsYSBJQSBlbiBlbCBkaWFnbsOzc3RpY28uXFxuKiAqKlJlc3BvbnNhYmlsaWRhZDoqKiAgRXN0YWJsZWNlciBjbGFyYW1lbnRlIGxhIHJlc3BvbnNhYmlsaWRhZCBkZWwgbcOpZGljbyBlbiBlbCBkaWFnbsOzc3RpY28geSB0cmF0YW1pZW50by5cXG4qICoqUHJpdmFjaWRhZDoqKiAgQ3VtcGxpciBjb24gbGFzIHJlZ3VsYWNpb25lcyBkZSBwcml2YWNpZGFkIGRlIGRhdG9zLlxcbiogKipQcmVjaXNpw7NuOioqICBHYXJhbnRpemFyIGxhIHByZWNpc2nDs24gZGUgbGEgaW5mb3JtYWNpw7NuIHByb3BvcmNpb25hZGEgcG9yIGxhIGFwbGljYWNpw7NuLlxcblxcblxcbk1lZGlBc3Npc3Qgc2UgY29uY2liZSBjb21vIHVuYSBwbGF0YWZvcm1hIGVuIGNvbnN0YW50ZSBldm9sdWNpw7NuLCBjb24gYWN0dWFsaXphY2lvbmVzIHJlZ3VsYXJlcyB5IGxhIGFkaWNpw7NuIGRlIG51ZXZhcyBmdW5jaW9uYWxpZGFkZXMgYmFzYWRhcyBlbiBsYSByZXRyb2FsaW1lbnRhY2nDs24gZGUgdXN1YXJpb3MgeSBhdmFuY2VzIHRlY25vbMOzZ2ljb3MuICBTdSBvYmpldGl2byBlcyBmYWNpbGl0YXIgbGEgYXRlbmNpw7NuIG3DqWRpY2EsIG1lam9yYXIgbGEgZWZpY2llbmNpYSB5IGZvcnRhbGVjZXIgbGEgcmVsYWNpw7NuIGVudHJlIG3DqWRpY29zIHkgcGFjaWVudGVzLiAgTGEgaW50ZWdyYWNpw7NuIGRlIGxhIElBIGRlYmUgc2VyIHNpZW1wcmUgcmVzcG9uc2FibGUgeSDDqXRpY2EsIGFjdHVhbmRvIGNvbW8gdW5hIGhlcnJhbWllbnRhIGRlIGFwb3lvIGFsIHByb2Zlc2lvbmFsIG3DqWRpY28sIG5vIGNvbW8gdW4gcmVlbXBsYXpvLlxcbiJ9XX1d' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "realiza la mejor app de asistencia medica para doctores y pacientes que puedas crear contempla estos requerimientos y anexa las mejores apps y funciones que tengas en tus sistemas e integralos para que sean funcionales entre si \\nConsultas m\u00e9dicas preliminares.\\nRespuestas autom\u00e1ticas a s\u00edntomas comunes.\\nAgendamiento de citas.\\nIntegraci\u00f3n con herramientas de IA para diagn\u00f3sticos"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "##  MediAssist: La Aplicaci\u00f3n Integral de Asistencia M\u00e9dica\\n\\nMediAssist es una aplicaci\u00f3n m\u00f3vil dise\u00f1ada para mejorar la experiencia de atenci\u00f3n m\u00e9dica tanto para m\u00e9dicos como para pacientes, integrando consultas preliminares, respuestas autom\u00e1ticas a s\u00edntomas, agendamiento de citas y herramientas de IA para diagn\u00f3s

## Call `generate_content`

In [2]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

**Apps y Funciones Integradas (ejemplos):**

* **Integración con Google Cloud Healthcare API:**  Para el análisis de imágenes médicas y procesamiento de datos de salud.
* **Integración con Amazon Comprehend Medical:**  Para la extracción de información relevante de informes médicos y textos.
* **Integración con Dialogflow o similar:**  Para construir el chatbot de respuestas automáticas a síntomas.
* **Integración con Stripe o PayPal:**  Para procesar pagos de consultas.


Esta descripción detalla la idea principal de la aplicación. Un desarrollo real requeriría un equipo multidisciplinario, un extenso plan de desarrollo, y una considerable inversión de tiempo y recursos.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")